Radek posted about this [here](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/discussion/309220), and linked to a GitHub repo with the code.

I just transferred that code here to Kaggle notebooks, that's all.

In [1]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# https://www.kaggle.com/c/h-and-m-personalized-fashion-recommendations/discussion/308635
def customer_hex_id_to_int(series):
    return series.str[-16:].apply(hex_id_to_int)

def hex_id_to_int(str):
    return int(str[-16:], 16)

def article_id_str_to_int(series):
    return series.astype('int32')

def article_id_int_to_str(series):
    return '0' + series.astype('str')

class Categorize(BaseEstimator, TransformerMixin):
    def __init__(self, min_examples=0):
        self.min_examples = min_examples
        self.categories = []
        
    def fit(self, X):
        for i in range(X.shape[1]):
            vc = X.iloc[:, i].value_counts()
            self.categories.append(vc[vc > self.min_examples].index.tolist())
        return self

    def transform(self, X):
        data = {X.columns[i]: pd.Categorical(X.iloc[:, i], categories=self.categories[i]).codes for i in range(X.shape[1])}
        return pd.DataFrame(data=data)


def calculate_apk(list_of_preds, list_of_gts):
    # for fast validation this can be changed to operate on dicts of {'cust_id_int': [art_id_int, ...]}
    # using 'data/val_week_purchases_by_cust.pkl'
    apks = []
    for preds, gt in zip(list_of_preds, list_of_gts):
        apks.append(apk(gt, preds, k=12))
    return np.mean(apks)

def eval_sub(sub_csv, skip_cust_with_no_purchases=True):
    sub=pd.read_csv(sub_csv)
    validation_set=pd.read_parquet('data/validation_ground_truth.parquet')

    apks = []

    no_purchases_pattern = []
    for pred, gt in zip(sub.prediction.str.split(), validation_set.prediction.str.split()):
        if skip_cust_with_no_purchases and (gt == no_purchases_pattern): continue
        apks.append(apk(gt, pred, k=12))
    return np.mean(apks)

In [3]:
import pandas as pd

In [4]:
%%time

transactions = pd.read_parquet('data/parquet/transactions_train.parquet')
customers = pd.read_parquet('data/parquet/customers.parquet')
articles = pd.read_parquet('data/parquet/articles.parquet')

# sample = 0.05
# transactions = pd.read_parquet(f'data/transactions_train_sample_{sample}.parquet')
# customers = pd.read_parquet(f'data/customers_sample_{sample}.parquet')
# articles = pd.read_parquet(f'data/articles_train_sample_{sample}.parquet')

CPU times: total: 3.72 s
Wall time: 1.72 s


In [5]:
test_week = transactions.week.max() + 1
transactions = transactions[transactions.week > transactions.week.max() - 10]

In [6]:
transactions = transactions[transactions['customer_id'].isin(transactions['customer_id'].unique()[:100000])]

In [7]:
transactions.week.min()

95

# Generating candidates

### Last purchase candidates

In [8]:
%%time

c2weeks = transactions.groupby('customer_id')['week'].unique()

CPU times: total: 6.05 s
Wall time: 6.16 s


In [9]:
transactions.groupby('week')['t_dat'].agg(['min', 'max'])

,min,max
week,,
95,2020-07-15,2020-07-21
96,2020-07-22,2020-07-28
97,2020-07-29,2020-08-04
98,2020-08-05,2020-08-11
99,2020-08-12,2020-08-18
100,2020-08-19,2020-08-25
101,2020-08-26,2020-09-01
102,2020-09-02,2020-09-08
103,2020-09-09,2020-09-15


In [10]:
c2weeks

customer_id
28847241659200          [95, 96, 101, 102]
200292573348128          [95, 96, 99, 102]
248294615847351                       [96]
272412481300040              [95, 96, 103]
375055163245029                   [96, 98]
                               ...        
18445903880167013488      [95, 96, 97, 98]
18445974752901854240                  [95]
18446007361892599794                  [96]
18446571879212697038              [95, 98]
18446624797007271432                  [95]
Name: week, Length: 100000, dtype: object

In [11]:
%%time

c2weeks2shifted_weeks = {}

for c_id, weeks in c2weeks.items():
    c2weeks2shifted_weeks[c_id] = {}
    for i in range(weeks.shape[0]-1):
        c2weeks2shifted_weeks[c_id][weeks[i]] = weeks[i+1]
    c2weeks2shifted_weeks[c_id][weeks[-1]] = test_week

CPU times: total: 391 ms
Wall time: 396 ms


In [12]:
c2weeks2shifted_weeks[28847241659200]

{95: 96, 96: 101, 101: 102, 102: 105}

In [13]:
candidates_last_purchase = transactions.copy()

In [14]:
%%time

weeks = []
for i, (c_id, week) in enumerate(zip(transactions['customer_id'], transactions['week'])):
    weeks.append(c2weeks2shifted_weeks[c_id][week])
    
candidates_last_purchase.week=weeks

CPU times: total: 8.31 s
Wall time: 8.42 s


In [15]:
candidates_last_purchase[candidates_last_purchase['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,96
29030504,2020-07-15,272412481300040,816592008,0.016932,1,96
29030505,2020-07-15,272412481300040,621381021,0.033881,1,96
29030506,2020-07-15,272412481300040,817477003,0.025407,1,96
29030507,2020-07-15,272412481300040,899088002,0.025407,1,96
29319533,2020-07-22,272412481300040,885077001,0.008458,1,103
29410772,2020-07-24,272412481300040,850176003,0.029034,2,103
29410773,2020-07-24,272412481300040,875803001,0.064559,2,103
29410774,2020-07-24,272412481300040,892970003,0.020966,2,103
29410775,2020-07-24,272412481300040,854619003,0.020966,2,103


In [16]:
transactions[transactions['customer_id']==272412481300040]

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95
29319533,2020-07-22,272412481300040,885077001,0.008458,1,96
29410772,2020-07-24,272412481300040,850176003,0.029034,2,96
29410773,2020-07-24,272412481300040,875803001,0.064559,2,96
29410774,2020-07-24,272412481300040,892970003,0.020966,2,96
29410775,2020-07-24,272412481300040,854619003,0.020966,2,96


### Bestsellers candidates

In [17]:
mean_price = transactions \
    .groupby(['week', 'article_id'])['price'].mean()

In [18]:
mean_price

week  article_id
95    108775015     0.004729
      108775044     0.008458
      110065001     0.006085
      110065002     0.006085
      111565001     0.004288
                      ...   
104   952267001     0.015390
      952938001     0.043203
      953450001     0.016932
      953763001     0.022017
      956217002     0.059305
Name: price, Length: 130832, dtype: float32

In [19]:
sales = transactions \
    .groupby('week')['article_id'].value_counts() \
    .groupby('week').rank(method='dense', ascending=False) \
    .groupby('week').head(12).rename('bestseller_rank').astype('int8')

In [20]:
sales

week  article_id
95    760084003      1
      866731001      2
      600886001      3
      706016001      4
      372860002      5
                    ..
104   929275001      6
      934835001      7
      919273002      8
      915529005      9
      909059002     10
Name: bestseller_rank, Length: 120, dtype: int8

In [21]:
sales.loc[95]

article_id
760084003     1
866731001     2
600886001     3
706016001     4
372860002     5
610776002     6
877278002     7
547780003     8
817354001     9
827968001    10
866731003    11
866383006    12
Name: bestseller_rank, dtype: int8

In [22]:
bestsellers_previous_week = pd.merge(sales, mean_price, on=['week', 'article_id']).reset_index()
bestsellers_previous_week.week += 1

In [23]:
bestsellers_previous_week.pipe(lambda df: df[df['week']==96])

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193
5,96,610776002,6,0.008318
6,96,877278002,7,0.025036
7,96,547780003,8,0.024814
8,96,817354001,9,0.021913
9,96,827968001,10,0.016436


In [24]:
unique_transactions = transactions \
    .groupby(['week', 'customer_id']) \
    .head(1) \
    .drop(columns=['article_id', 'price']) \
    .copy()

In [25]:
unique_transactions

,t_dat,customer_id,sales_channel_id,week
29030503,2020-07-15,272412481300040,1,95
29064059,2020-07-15,1456826891333599,1,95
29067103,2020-07-15,2133687643102426,2,95
29027487,2020-07-15,6010692573790711,1,95
29046403,2020-07-15,6171059100114610,2,95
...,...,...,...,...
31777562,2020-09-22,18384093855814512002,2,104
31776339,2020-09-22,18394381115614748074,2,104
31778354,2020-09-22,18402021132185235134,2,104
31773475,2020-09-22,18416046886745463528,2,104


In [26]:
transactions.drop_duplicates(['week', 'customer_id'])

,t_dat,customer_id,article_id,price,sales_channel_id,week
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95
29064059,2020-07-15,1456826891333599,888294001,0.013542,1,95
29067103,2020-07-15,2133687643102426,843642001,0.042356,2,95
29027487,2020-07-15,6010692573790711,857812010,0.039661,1,95
29046403,2020-07-15,6171059100114610,815447007,0.006763,2,95
...,...,...,...,...,...,...
31777562,2020-09-22,18384093855814512002,889669006,0.052966,2,104
31776339,2020-09-22,18394381115614748074,901318001,0.101678,2,104
31778354,2020-09-22,18402021132185235134,918516001,0.050831,2,104
31773475,2020-09-22,18416046886745463528,903420001,0.030864,2,104


In [27]:
candidates_bestsellers = pd.merge(
    unique_transactions,
    bestsellers_previous_week,
    on='week',
)

In [28]:
test_set_transactions = unique_transactions.drop_duplicates('customer_id').reset_index(drop=True)
test_set_transactions.week = test_week

In [29]:
test_set_transactions

,t_dat,customer_id,sales_channel_id,week
0,2020-07-15,272412481300040,1,105
1,2020-07-15,1456826891333599,1,105
2,2020-07-15,2133687643102426,2,105
3,2020-07-15,6010692573790711,1,105
4,2020-07-15,6171059100114610,2,105
...,...,...,...,...
99995,2020-07-24,2887856003372290412,1,105
99996,2020-07-24,2895454695095217046,1,105
99997,2020-07-24,2896143988595837838,2,105
99998,2020-07-24,2896624638413361746,1,105


In [30]:
candidates_bestsellers_test_week = pd.merge(
    test_set_transactions,
    bestsellers_previous_week,
    on='week'
)

In [31]:
candidates_bestsellers = pd.concat([candidates_bestsellers, candidates_bestsellers_test_week])
candidates_bestsellers.drop(columns='bestseller_rank', inplace=True)

In [32]:
candidates_bestsellers

,t_dat,customer_id,sales_channel_id,week,article_id,price
0,2020-07-22,200292573348128,2,96,760084003,0.025094
1,2020-07-22,200292573348128,2,96,866731001,0.024919
2,2020-07-22,200292573348128,2,96,600886001,0.022980
3,2020-07-22,200292573348128,2,96,706016001,0.033197
4,2020-07-22,200292573348128,2,96,372860002,0.013193
...,...,...,...,...,...,...
1199995,2020-07-24,2897589988856734901,1,105,929275001,0.058097
1199996,2020-07-24,2897589988856734901,1,105,934835001,0.024667
1199997,2020-07-24,2897589988856734901,1,105,919273002,0.041821
1199998,2020-07-24,2897589988856734901,1,105,915529005,0.033458


# Age Group Candidates

In [33]:
# t_data = pd.merge(transactions, customers, on='customer_id', how='left')

In [34]:
# t_data.head()

In [35]:
# t_data['age_group'] = pd.cut(t_data['age'], bins=[0, 25, 40, 60, 100], labels=[0, 1, 2, 3])

In [36]:
# t_data.head()

In [37]:
# unique_t_data = t_data \
#     .groupby(['week', 'customer_id']) \
#     .head(1) \
#     .drop(columns=['article_id', 'price']) \
#     .copy()

In [38]:
# sales_by_age_group = t_data \
#     .groupby('age_group')['article_id'].value_counts() \
#     .groupby('age_group').rank(method='dense', ascending=False) \
#     .groupby('age_group').head(12).rename('bestseller_agegroup_rank').astype('int8')

In [39]:
# sales_by_age_group

In [40]:
# candidates_bestsellers = pd.merge(
#     unique_transactions,
#     bestsellers_agegroup_rank,
#     on='age_group',
# )

In [41]:
# candidates_bestsellers.head()

# Combining transactions and candidates / negative examples

In [42]:
transactions['purchased'] = 1

In [43]:
data = pd.concat([transactions, candidates_last_purchase, candidates_bestsellers])
data.purchased.fillna(0, inplace=True)

In [44]:
data

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased
29030503,2020-07-15,272412481300040,778064028,0.008458,1,95,1.0
29030504,2020-07-15,272412481300040,816592008,0.016932,1,95,1.0
29030505,2020-07-15,272412481300040,621381021,0.033881,1,95,1.0
29030506,2020-07-15,272412481300040,817477003,0.025407,1,95,1.0
29030507,2020-07-15,272412481300040,899088002,0.025407,1,95,1.0
...,...,...,...,...,...,...,...
1199995,2020-07-24,2897589988856734901,929275001,0.058097,1,105,0.0
1199996,2020-07-24,2897589988856734901,934835001,0.024667,1,105,0.0
1199997,2020-07-24,2897589988856734901,919273002,0.041821,1,105,0.0
1199998,2020-07-24,2897589988856734901,915529005,0.033458,1,105,0.0


In [45]:
data.drop_duplicates(['customer_id', 'article_id', 'week'], inplace=True)

In [46]:
data.purchased.mean()

0.17035463761924743

### Add bestseller information

In [47]:
data = pd.merge(
    data,
    bestsellers_previous_week[['week', 'article_id', 'bestseller_rank']],
    on=['week', 'article_id'],
    how='left'
)

In [48]:
bestsellers_previous_week.head()

,week,article_id,bestseller_rank,price
0,96,760084003,1,0.025094
1,96,866731001,2,0.024919
2,96,600886001,3,0.022980
3,96,706016001,4,0.033197
4,96,372860002,5,0.013193


In [49]:
data = data[data.week != data.week.min()]
data.bestseller_rank.fillna(999, inplace=True)

In [50]:
agg_price_spent = transactions.groupby('customer_id')['price'].agg(['mean', 'max']).reset_index()
agg_price_spent.rename(columns={'mean': 'avg_price_spent', 'max': 'max_price_spent'}, inplace=True)

In [51]:
agg_price_spent

,customer_id,avg_price_spent,max_price_spent
0,28847241659200,0.046085,0.128797
1,200292573348128,0.029602,0.064305
2,248294615847351,0.022017,0.031458
3,272412481300040,0.030597,0.067780
4,375055163245029,0.020322,0.033881
...,...,...,...
99995,18445903880167013488,0.021818,0.067780
99996,18445974752901854240,0.025407,0.025407
99997,18446007361892599794,0.029644,0.033881
99998,18446571879212697038,0.025956,0.050831


In [52]:
transactions_with_age = pd.merge(transactions, customers[['customer_id', 'age']], on='customer_id', how='left')
avg_purchaser_age = transactions_with_age.groupby('article_id')['age'].mean().reset_index()
avg_purchaser_age.rename(columns={'age': 'avg_purchaser_age'}, inplace=True)

In [53]:
avg_purchaser_age

,article_id,avg_purchaser_age
0,108775015,40.333333
1,108775044,40.727273
2,110065001,18.000000
3,110065002,21.000000
4,111565001,31.157895
...,...,...
31854,952267001,42.444444
31855,952938001,27.333333
31856,953450001,28.000000
31857,953763001,39.125000


In [54]:
customers = pd.merge(customers, agg_price_spent, on='customer_id', how='left')
articles = pd.merge(articles, avg_purchaser_age, on='article_id', how='left')

In [55]:
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc,avg_purchaser_age
0,108775015,108775,12855,253,9,0,1010016,0,9,0,...,0,0,1,0,16,30,1002,2,8834,40.333333
1,108775044,108775,12855,253,9,0,1010016,0,10,2,...,0,0,1,0,16,30,1002,2,8834,40.727273
2,108775051,108775,44846,253,9,0,1010017,3,11,11,...,0,0,1,0,16,30,1002,2,8834,NaN
3,110065001,110065,8159,306,13,4,1010016,0,9,0,...,7,7,1,0,61,5,1017,4,8243,18.000000
4,110065002,110065,8159,306,13,4,1010016,0,10,2,...,7,7,1,0,61,5,1017,4,8243,21.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,24043,302,14,7,1010014,6,9,0,...,2,2,3,3,26,16,1021,13,28026,28.000000
105538,953763001,953763,24027,253,9,0,1010016,0,9,0,...,0,0,1,0,2,15,1005,0,28025,39.125000
105539,956217002,956217,24040,265,1,2,1010016,0,9,0,...,0,0,1,0,18,12,1005,0,28024,44.666667
105540,957375001,957375,24041,72,48,3,1010016,0,9,0,...,1,1,2,2,52,25,1019,1,28023,NaN


In [56]:
articles.avg_purchaser_age.fillna(-1, inplace=True)

In [57]:
data = pd.merge(data, articles, on='article_id', how='left')
data = pd.merge(data, customers, on='customer_id', how='left')

In [58]:
articles['index_code'].drop_duplicates()

0       0
3       7
11      5
16      2
28      6
40      9
69      3
78      1
81      4
1367    8
Name: index_code, dtype: int8

In [59]:
pivot_table = pd.pivot_table(
    data,
    index='customer_id',
    columns='index_code',
    values='article_id',
    aggfunc='count',
    fill_value=0
)

pivot_table['total_purchases'] = pivot_table.sum(axis=1)

# Calculate the percentage of purchases in each category for each customer
pivot_table['percentage_women_purchases'] = ((pivot_table[0] + pivot_table[7] + pivot_table[6]) / pivot_table['total_purchases'])
pivot_table['percentage_children_purchases'] = ((pivot_table[5] + pivot_table[3] + pivot_table[4] + pivot_table[8]) / pivot_table['total_purchases'])
pivot_table['percentage_men_purchases'] = (pivot_table[2] / pivot_table['total_purchases'])

In [60]:
pivot_table.reset_index(inplace=True)
pivot_table

index_code,customer_id,0,1,2,3,4,5,6,7,8,9,total_purchases,percentage_women_purchases,percentage_children_purchases,percentage_men_purchases
0,28847241659200,38,10,0,0,0,0,0,3,0,5,56,0.732143,0.0,0.000000
1,200292573348128,45,12,1,0,0,0,0,12,0,4,74,0.770270,0.0,0.013514
2,248294615847351,12,7,8,0,0,0,0,3,0,2,32,0.468750,0.0,0.250000
3,272412481300040,32,14,0,0,0,0,2,11,0,2,61,0.737705,0.0,0.000000
4,375055163245029,33,22,0,0,0,0,0,5,0,2,62,0.612903,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,18445903880167013488,60,28,0,0,0,0,18,20,0,4,130,0.753846,0.0,0.000000
99996,18445974752901854240,9,4,0,0,0,0,0,0,0,0,13,0.692308,0.0,0.000000
99997,18446007361892599794,16,7,0,0,0,0,0,3,0,2,28,0.678571,0.0,0.000000
99998,18446571879212697038,24,7,1,0,0,0,1,7,0,0,40,0.800000,0.0,0.025000


In [61]:
data = pd.merge(data, pivot_table[['customer_id', 'total_purchases', 'percentage_women_purchases', 'percentage_children_purchases', 'percentage_men_purchases']], on='customer_id', how='left')

In [62]:
data.sort_values(['week', 'customer_id'], inplace=True)
data.reset_index(drop=True, inplace=True)

In [63]:
train = data[data.week != test_week]
test = data[data.week==test_week].drop_duplicates(['customer_id', 'article_id', 'sales_channel_id']).copy()

In [89]:
train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,week,purchased,bestseller_rank,product_code,prod_name,...,club_member_status,fashion_news_frequency,age,postal_code,avg_price_spent,max_price_spent,total_purchases,percentage_women_purchases,percentage_children_purchases,percentage_men_purchases
0,2020-07-26,28847241659200,887770001,0.016932,1,96,1.0,999.0,887770,727,...,0,1,21,57896,0.046085,0.128797,56,0.732143,0.0,0.0
1,2020-07-18,28847241659200,762846001,0.025407,1,96,0.0,999.0,762846,472,...,0,1,21,57896,0.046085,0.128797,56,0.732143,0.0,0.0
2,2020-07-18,28847241659200,829308001,0.033881,1,96,0.0,999.0,829308,11402,...,0,1,21,57896,0.046085,0.128797,56,0.732143,0.0,0.0
3,2020-07-26,28847241659200,760084003,0.025094,1,96,0.0,1.0,760084,1134,...,0,1,21,57896,0.046085,0.128797,56,0.732143,0.0,0.0
4,2020-07-26,28847241659200,866731001,0.024919,1,96,0.0,2.0,866731,3609,...,0,1,21,57896,0.046085,0.128797,56,0.732143,0.0,0.0


In [65]:
train.size

133097805

In [66]:
train_baskets = train.groupby(['week', 'customer_id'])['article_id'].count().values

In [67]:
columns_to_use = ['article_id', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id',
'perceived_colour_master_id', 'department_no', 'index_code',
'index_group_no', 'section_no', 'garment_group_no', 'FN', 'Active',
'club_member_status', 'fashion_news_frequency', 'age', 'postal_code', 'bestseller_rank']

In [68]:
%%time

train_X = train[columns_to_use]
train_y = train['purchased']

test_X = test[columns_to_use]

CPU times: total: 156 ms
Wall time: 170 ms


# Model training

In [69]:
from lightgbm.sklearn import LGBMRanker

In [70]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=10
)

In [71]:
%%time

ranker = ranker.fit(
    train_X,
    train_y,
    group=train_baskets,
)

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.847697
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.152754
[LightGBM] [Debug] init for col-wise cost 0.094621 seconds, init for row-wise cost 0.221427 seconds
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.283212 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1085
[LightGBM] [Info] Number of data points in the train set: 2957729, number of used features: 18
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
CPU times: total: 9.42 s
Wall time: 2.2 s


In [72]:
for i in ranker.feature_importances_.argsort()[::-1]:
    print(columns_to_use[i], ranker.feature_importances_[i]/ranker.feature_importances_.sum())

bestseller_rank 0.9970490950649039
article_id 0.0010892316088210255
product_type_no 0.000795982917164999
garment_group_no 0.0007666139842055402
age 0.0001026063081180252
department_no 9.235051830224613e-05
graphical_appearance_no 6.803979985224442e-05
section_no 3.6079798631995007e-05
fashion_news_frequency 0.0
club_member_status 0.0
Active 0.0
FN 0.0
postal_code 0.0
index_code 0.0
perceived_colour_master_id 0.0
perceived_colour_value_id 0.0
colour_group_code 0.0
index_group_no 0.0


# Calculate predictions

In [73]:
%time

test['preds'] = ranker.predict(test_X)

c_id2predicted_article_ids = test \
    .sort_values(['customer_id', 'preds'], ascending=False) \
    .groupby('customer_id')['article_id'].apply(list).to_dict()

bestsellers_last_week = \
    bestsellers_previous_week[bestsellers_previous_week.week == bestsellers_previous_week.week.max()]['article_id'].tolist()

CPU times: total: 0 ns
Wall time: 0 ns


# Create submission

In [74]:
sub = pd.read_csv('data/original/sample_submission.csv')

In [75]:
%%time
preds = []
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    preds.append(pred[:12])

CPU times: total: 3.83 s
Wall time: 3.89 s


In [76]:
preds = [' '.join(['0' + str(p) for p in ps]) for ps in preds]
sub.prediction = preds

In [77]:
sub_name = 'baseline_model'
sub.to_csv(f'{sub_name}.csv.gz', index=False)

# Evaluate candidate generation methods

In [78]:
%%time
final_preds = {}
for c_id in customer_hex_id_to_int(sub.customer_id):
    pred = c_id2predicted_article_ids.get(c_id, [])
    pred = pred + bestsellers_last_week
    final_preds[c_id] = pred[:12]

CPU times: total: 3.55 s
Wall time: 3.56 s


In [79]:
def precision_at_K(true_candidates, predicted_candidates, K):
    relevant_candidates = set(true_candidates)
    top_K_predicted = predicted_candidates[:K]
    relevant_predicted = [c for c in top_K_predicted if c in relevant_candidates]
    return len(relevant_predicted) / K

In [80]:
def precision(true_candidates, predicted_candidates, K):
    relevant_candidates = set(true_candidates)
    top_K_predicted = predicted_candidates
    relevant_predicted = [c for c in top_K_predicted if c in relevant_candidates]
    return len(relevant_predicted) / K

In [81]:
grouped_candidates = candidates_bestsellers.groupby('customer_id')['article_id'].agg(list).reset_index()

In [82]:
grouped_candidates.head()

,customer_id,article_id
0,28847241659200,"[760084003, 866731001, 600886001, 706016001, 3..."
1,200292573348128,"[760084003, 866731001, 600886001, 706016001, 3..."
2,248294615847351,"[760084003, 866731001, 600886001, 706016001, 3..."
3,272412481300040,"[760084003, 866731001, 600886001, 706016001, 3..."
4,375055163245029,"[760084003, 866731001, 600886001, 706016001, 3..."


In [83]:
candidates_bestsellers[candidates_bestsellers['customer_id'] == 11246327431398957306]

,t_dat,customer_id,sales_channel_id,week,article_id,price


In [84]:
grouped_candidates[grouped_candidates['customer_id'] == 11246327431398957306]

,customer_id,article_id


In [85]:
final_preds[11246327431398957306]

[924243001,
 909370001,
 924243002,
 918522001,
 935541001,
 923758001,
 889550002,
 929275001,
 934835001,
 919273002,
 915529005,
 909059002]

In [86]:
grouped_dict = dict(zip(grouped_candidates['customer_id'], grouped_candidates['article_id']))
for key, item in grouped_dict.items():
    print(key, item)
    break

28847241659200 [760084003, 866731001, 600886001, 706016001, 372860002, 610776002, 877278002, 547780003, 817354001, 827968001, 866731003, 866383006, 916468003, 896152003, 896152002, 920610001, 921906003, 915523001, 751471043, 915526002, 915529003, 877769001, 918292001, 933891001, 933706001, 898694001, 915529003, 751471001, 915526001, 915526002, 751471043, 915529001, 916468003, 863583002, 863595006, 918292001, 924243001, 909370001, 924243002, 918522001, 935541001, 923758001, 889550002, 929275001, 934835001, 919273002, 915529005, 909059002]


In [87]:
from tqdm import tqdm
from collections import defaultdict

# Calculate P@12 for each user and each candidate generation method
results = {}

for method, generated_candidates in [('Recently Bought', candidates_last_purchase), ('Bestsellers', candidates_bestsellers), ('Bestsellers test week', candidates_bestsellers_test_week)]:
    grouped_candidates = generated_candidates.groupby('customer_id')['article_id'].agg(list).reset_index()
    grouped_dict = defaultdict(list, zip(grouped_candidates['customer_id'], grouped_candidates['article_id']))
    method_results = []
    for user, top_12 in tqdm(final_preds.items()):
        precision_at_12 = precision(top_12, grouped_dict[user], K=12)
        method_results.append(precision_at_12)
    results[method] = method_results

# Calculate the average P@12 for each method
average_precision = {method: sum(method_results) / len(method_results) for method, method_results in results.items()}

# Print the results
for method, avg_precision in average_precision.items():
    print(f"Method: {method}, Average P@12: {avg_precision}")

100%|██████████| 1371980/1371980 [00:03<00:00, 354650.39it/s]

Method: Recently Bought, Average P@12: 0.021160779797564793
Method: Bestsellers, Average P@12: 0.057046324776357
Method: Bestsellers test week, Average P@12: 0.05433661569410408
